# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
%%capture output --no-stderr

# Install async module
!uv pip install nest_asyncio

In [2]:
# Import the SDK
import syft_nsai_sdk as sdk
import nest_asyncio
import httpx
import logging
import os

In [3]:
# Import client
from syft_nsai_sdk import Client
from syft_nsai_sdk.utils.logger import get_logger

In [4]:
# define utilities
nest_asyncio.apply()
logger = get_logger()
logging.getLogger('syft_nsai_sdk').setLevel(logging.INFO)
logger.success("Logger initialized successfully!")

2025-09-05 01:50:26 - syft_nsai_sdk - SUCCESS - Logger initialized successfully!


In [5]:
# Initialize client
client = Client()

2025-09-05 01:50:27 - syft_nsai_sdk.core.config - INFO - Using config credentials at path: /Users/callisezenwaka/.syftbox/config.json
2025-09-05 01:50:27 - syft_nsai_sdk.main - INFO - Found existing accounting credentials for callis@openmined.org
2025-09-05 01:50:27 - syft_nsai_sdk.main - INFO - Client initialized for callis@openmined.org


## Basic usage

In [6]:
# inline service usage - , "bob@example.com/wiki"
inline_result = client.pipeline(
    data_sources=["callis@openmined.org/carl-model", "callis@openmined.org/carl-free"], 
    synthesizer="callis@openmined.org/carl-model"
).run(messages=[{"role": "user", "content": "What is Python?"}])

✓ Response received                                                             



In [7]:
# show response
logger.info(f"Pipeline result:\n\n {inline_result}")

2025-09-05 01:50:40 - syft_nsai_sdk - INFO - Pipeline result:

 PyTorch is a deep learning library for Python developed by PyTorch Foundation, an open-source organization. It provides a platform for researchers and data scientists to develop machine learning algorithms and models in a fast, flexible, and easy-to-use way. PyTorch offers features such as automatic differentiation (AD), distributed training, high-performance computing (HPC) support, and various optimizers, including Adam and SGD.

Key features of PyTorch include:

1. Automatic Differentiation: This feature enables PyTorch to automatically differentiate the users' code during training, reducing the need for manual grading or programming.

2. Distributed Training: PyTorch supports distributed training on various HPC platforms such as NVIDIA Tesla and AMD Vulcan. It can also be used in a non-distributed mode for single node training.

3. High-Performance Computing (HPC) Support: PyTorch is highly optimized for high-performan

In [8]:
# Object-Oriented service usage
pipeline = client.create_pipeline()
pipeline.add_source("callis@openmined.org/sota-free", topK=8)
pipeline.add_source("callis@openmined.org/carl-model", topK=8)
pipeline.set_synthesizer("callis@openmined.org/sota-free", temperature=0.7)  
oop_result = pipeline.run(messages=[{"role": "user", "content": "What is Python?"}])

✓ Response received                                                             



In [9]:
# show response
logger.info(f"Pipeline result:\n\n {oop_result}")

2025-09-05 01:50:52 - syft_nsai_sdk - INFO - Pipeline result:

 Python (pronounced "py-then") is a general-purpose programming language developed by Python Software Foundation. It was originally designed for server computing, but has since gained popularity in the world of web development and data analysis. Its syntax and features are similar to those of other high-level programming languages like C++, Java, and C#, making it a versatile choice for a variety of applications. Here are some key features of Python:

1. Easy learning curve: Python is relatively easy to learn due to its intuitive syntax and clear structure. It's designed to be beginner-friendly and accessible to anyone with familiarity with Python, while still allowing advanced users to take advantage of the language's powerful features.

2. Large community: Python has a large and active community, with multiple libraries, communities, and forums available for users to explore and connect with other developers. This helps t

## Advanced usage

In [10]:
# Load some services as objects first
carl_model = client.load("callis@openmined.org/carl-model") # paid
sota_free = client.load("callis@openmined.org/sota-free") # free
carl_free = client.load("callis@openmined.org/carl-free") # free
carl_claude = client.load("callis@openmined.org/carl-claude") # free

In [11]:
# Demo mixed input types
mixed_result = client.pipeline_mixed(
    data_sources=[
        "callis@openmined.org/carl_model",           # String
        carl_free,                                       # Service object
        {"name": "callis@openmined.org/carl-claude", "topK": 5},  # Dict with string
        {"name": carl_model, "topK": 8}                   # Dict with Service object
    ],
    synthesizer=carl_model  # Service object as synthesizer
).run(messages=[{"role": "user", "content": "What is Python?"}])

2025-09-05 01:53:02 - syft_nsai_sdk.core.pipeline - ERROR - Search failed for callis@openmined.org/carl-free: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received

                                                                                

2025-09-05 01:53:03 - syft_nsai_sdk.core.pipeline - ERROR - Search failed for callis@openmined.org/sota-free: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-05 01:53:03 - syft_nsai_sdk.core.pipeline - WARNING - Search failed for source callis@openmined.org/sota-free: RPC_ERROR: Polling timed out after 30/30 attempts
2025-09-05 01:53:03 - syft_nsai_sdk.core.pipeline - WARNING - Search failed for source callis@openmined.org/carl-free: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received



ValidationError: VALIDATION_ERROR: All data source searches failed

In [ ]:
logger.info(f"Mixed pipeline result:\n\n {mixed_result}")

In [12]:
# Demo with pure Service objects
service_result = client.pipeline_mixed(
    data_sources=[sota_free, carl_free],
    synthesizer=sota_free
).run(messages=[{"role": "user", "content": "What is machine learning?"}])

✓ Response received                                                             



In [13]:
logger.info(f"Service objects pipeline result:\n\n {service_result}")

2025-09-05 01:53:57 - syft_nsai_sdk - INFO - Service objects pipeline result:

 Machine learning is a subfield of artificial intelligence that involves the development and application of algorithms to learn from data in order to make predictions or decisions. It combines aspects of statistics, computer science, and economics, and has applications across various fields such as finance, healthcare, transportation, and retail. Machine learning algorithms rely on large datasets to learn patterns and make predictions, often using machine learning techniques like decision trees, neural networks, and random forests. By leveraging this knowledge, machine learning algorithms can help solve complex problems, making it a powerful tool in many industries.


In [ ]:
# Demo Service objects with different parameters
param_result = client.pipeline_mixed(
    data_sources=[
        {"name": carl_free, "topK": 10},
        {"name": "callis@openmined.org/carl-claude", "topK": 3}
    ],
    synthesizer={"name": carl_model, "temperature": 0.9}
).run(messages=[{"role": "user", "content": "Explain neural networks"}])

⠇ Waiting for service response...

In [ ]:
logger.info(f"Parameters pipeline result:\n\n {param_result}")